In [303]:
import os
os.chdir('/LUSTRE/MADMEX/tasks/2019_tasks/app_label_segments/caribbean_git')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "caribbean.settings")

import django
django.setup()

import fiona

from datetime import datetime
from shapely.geometry import shape
from django.contrib.gis.geos import GEOSGeometry
from fiona.crs import to_string
from pyproj import Proj

from madmex.util.spatial import feature_transform
from caribbean.models import MadmexCaribesample

In [307]:
def ingest_feature_from_shp(shapefile) :
    with fiona.open(shapefile, encoding='utf-8') as src:
        fc = list(src)
        src_crs = src.crs
        to_string_crs = to_string(src_crs)
        proj_crs = Proj(src.crs)
        if not proj_crs.is_latlong():
            fc_proj = [feature_transform(x,"+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",to_string_crs) for x in fc]
        else:
            fc_proj = fc
            
        id_list = [feat['properties']['id'] for feat in src]
        cv31_list = [feat['properties']['clave_31'] for feat in src]
        area_list = [feat['properties']['area'] for feat in src]
        
    shape_list = [shape(feat['geometry']) for feat in fc_proj]
    
    for i in range(len(id_list)):
        confidence=0
        st_area=area_list[i]
        country_id=86
        tag_id=127 #cv31_list[i] 
        tag_editable_id=127 #cv31_list[i]
        validated=False
        added = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        geom = GEOSGeometry(shape_list[i].wkt,4326)
        _ = MadmexCaribesample.objects.get_or_create(the_geom=geom,
                                                      added=added,
                                                      confidence=confidence,
                                                      st_area=st_area,
                                                      country_id=country_id,
                                                      tag_id=tag_id,
                                                      tag_editable_id=tag_editable_id,
                                                      validated=validated)    

In [308]:
#shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/xalapa_inecol_taller/indi50k_veracruz/E15C47_madmex_sentinel2_2018_SUB_repro.shp'
shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/xalapa_inecol_taller/indi50k_veracruz/repros/F14B74_madmex_sentinel2_2018_repro.shp'

In [309]:
ingest_feature_from_shp(shapefile)

/usr/local/lib/python3.6/dist-packages/django/db/models/fields/__init__.py:1421: RuntimeWarning: DateTimeField MadmexCaribesample.added received a naive datetime (2019-07-12 20:29:13) while time zone support is active.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/django/db/models/fields/__init__.py:1421: RuntimeWarning: DateTimeField MadmexCaribesample.added received a naive datetime (2019-07-12 20:29:14) while time zone support is active.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/django/db/models/fields/__init__.py:1421: RuntimeWarning: DateTimeField MadmexCaribesample.added received a naive datetime (2019-07-12 20:29:15) while time zone support is active.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/django/db/models/fields/__init__.py:1421: RuntimeWarning: DateTimeField MadmexCaribesample.added received a naive datetime (2019-07-12 20:29:16) while time zone support is active.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/django/db/mod